## Datenquelle
http://www.offenedaten.frankfurt.de/dataset/frankfurter-stadtteilgrenzen-fur-gis-systeme

- imported into QGIS (https://www.qgis.org)
- exported as csv-Datei

In [1]:
%matplotlib inline

In [2]:
%%capture --no-display
!pip uninstall tensorflow protobuf --yes

In [3]:
%%capture --no-display
import pandas as pd
import numpy as np
import json
import requests
from pandas.io.json import json_normalize
!pip uninstall tensorflow-tensorboard -y
!pip install tensorflow geopy 
from geopy.geocoders import Nominatim
!pip install folium
import folium
from folium import plugins

In [4]:
# The code was removed by Watson Studio for sharing.

The geograpical coordinate of Altstadt are 50.11104545, 8.68293572698541.
The geograpical coordinate of Innenstadt are 50.11287765, 8.67492196213115.
The geograpical coordinate of Bahnhofsviertel are 50.10774145, 8.66873601221096.
The geograpical coordinate of Westend-Süd are 50.1152448, 8.6622696.
The geograpical coordinate of Westend-Nord are 50.1263563, 8.6679209.
The geograpical coordinate of Nordend-West are 50.1249136, 8.67795.
The geograpical coordinate of Nordend-Ost are 50.1249197, 8.6923167.
The geograpical coordinate of Ostend are 50.1159353, 8.72054636793775.
The geograpical coordinate of Bornheim are 50.13305575, 8.71493241229322.
The geograpical coordinate of Gutleutviertel are 50.0978128, 8.65076020625548.
The geograpical coordinate of Gallus are 50.10365845, 8.63670648468984.
The geograpical coordinate of Bockenheim are 50.12236065, 8.63753199693013.
The geograpical coordinate of Sachsenhausen-N. are 50.1073316, 8.6876717.
The geograpical coordinate of Sachsenhausen-S. 

,Stadtteil,Latitude,Longitude
0,Altstadt,50.111045,8.682936
1,Innenstadt,50.112878,8.674922
2,Bahnhofsviertel,50.107741,8.668736
3,Westend-Süd,50.115245,8.662270
4,Westend-Nord,50.126356,8.667921
5,Nordend-West,50.124914,8.677950
6,Nordend-Ost,50.124920,8.692317
7,Ostend,50.115935,8.720546
8,Bornheim,50.133056,8.714932
9,Gutleutviertel,50.097813,8.650760


In [5]:
# create map of Frankfurt using latitude and longitude values
map_ffm = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(df_ffm['Latitude'], df_ffm['Longitude'], df_ffm['Stadtteil']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=5,
        popup=label,
        tooltip=borough,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ffm)  
    
map_ffm

In [1]:
CLIENT_ID = 'KXIMMVCZIOQOGIFALLN3DBBMVLZH2IXV3XN5OQVOLN05S4VH' # your Foursquare ID
CLIENT_SECRET = '1FNPCQCB3ZGYRLA3AGYRCKL0WD0B2NAMH3GS3OC5N3KP1CXI' # your Foursquare Secret
VERSION = '20190528' # Foursquare API version

df_ffm[df_ffm['Stadtteil']=='Innenstadt']['Stadtteil']

In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    lat_list=[]
    lng_list=[]
    popup_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        lat_list = [(v['venue']['location']['lat']) for v in results]
        lng_list = [(v['venue']['location']['lng']) for v in results]
        marker_list= list(zip(lat_list,lng_list))
        
        popup_list = [(v['venue']['name']) for v in results]
        
        folium.plugins.MarkerCluster(
            locations=marker_list,
            popups=popup_list
            ).add_to(map_ffm)

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
LIMIT = 100
borough_venues = getNearbyVenues(names=df_ffm['Stadtteil'],
                                   latitudes=df_ffm['Latitude'],
                                   longitudes=df_ffm['Longitude']
                                  )
map_ffm

Altstadt
Innenstadt
Bahnhofsviertel
Westend-Süd
Westend-Nord
Nordend-West
Nordend-Ost
Ostend
Bornheim
Gutleutviertel
Gallus
Bockenheim
Sachsenhausen-N.
Sachsenhausen-S.
Flughafen
Oberrad
Niederrad
Schwanheim
Griesheim
Rödelheim
Hausen
Praunheim
Heddernheim
Niederursel
Ginnheim
Dornbusch
Eschersheim
Eckenheim
Preungesheim
Bonames
Berkersheim
Riederwald
Seckbach
Fechenheim
Höchst
Nied
Sindlingen
Zeilsheim
Unterliederbach
Sossenheim
Nieder-Erlenbach
Kalbach-Riedberg
Harheim
Nieder-Eschbach
Bergen-Enkheim
Frankfurter Berg


In [10]:
print(borough_venues.shape)
borough_venues.head()

(867, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Altstadt,50.111045,8.682936,Kleinmarkthalle,50.112778,8.682958,Market
1,Altstadt,50.111045,8.682936,Weinterasse Rollanderhof,50.112473,8.682164,Wine Bar
2,Altstadt,50.111045,8.682936,Römerberg,50.110489,8.682131,Plaza
3,Altstadt,50.111045,8.682936,SCHIRN Kunsthalle,50.110291,8.683542,Art Museum
4,Altstadt,50.111045,8.682936,Dom Aussichtsplattform,50.110609,8.684908,Scenic Lookout


In [11]:
borough_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Altstadt,100,100,100,100,100,100
Bahnhofsviertel,100,100,100,100,100,100
Bergen-Enkheim,1,1,1,1,1,1
Berkersheim,2,2,2,2,2,2
Bockenheim,24,24,24,24,24,24
Bonames,9,9,9,9,9,9
Bornheim,19,19,19,19,19,19
Dornbusch,6,6,6,6,6,6
Eckenheim,8,8,8,8,8,8


In [12]:
print('There are {} uniques categories.'.format(len(borough_venues['Venue Category'].unique())))

There are 189 uniques categories.


In [13]:
uvc = borough_venues['Venue Category'].unique()
print( sorted(uvc))

['ATM', 'African Restaurant', 'Airport Lounge', 'Airport Service', 'American Restaurant', 'Apple Wine Pub', 'Argentinian Restaurant', 'Art Gallery', 'Art Museum', 'Asian Restaurant', 'Austrian Restaurant', 'BBQ Joint', 'Bakery', 'Bank', 'Bar', 'Beach Bar', 'Bed & Breakfast', 'Beer Bar', 'Beer Garden', 'Beer Store', 'Big Box Store', 'Bistro', 'Board Shop', 'Boarding House', 'Bookstore', 'Boutique', 'Bowling Alley', 'Breakfast Spot', 'Brewery', 'Building', 'Burger Joint', 'Bus Station', 'Bus Stop', 'Butcher', 'Cafeteria', 'Café', 'Cajun / Creole Restaurant', 'Chinese Restaurant', 'Chocolate Shop', 'Church', 'Cigkofte Place', 'Clothing Store', 'Cocktail Bar', 'Coffee Shop', 'College Residence Hall', 'Concert Hall', 'Construction & Landscaping', 'Convenience Store', 'Cosmetics Shop', 'Currywurst Joint', 'Department Store', 'Dessert Shop', 'Diner', 'Discount Store', 'Dive Bar', 'Doner Restaurant', 'Donut Shop', 'Drugstore', 'Eastern European Restaurant', 'Electronics Store', 'Ethiopian Rest

In [14]:
# one hot encoding
borough_onehot = pd.get_dummies(borough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
borough_onehot['Neighbourhood'] = borough_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [borough_onehot.columns[-1]] + list(borough_onehot.columns[:-1])
borough_onehot = borough_onehot[fixed_columns]

borough_onehot.head()

,Neighbourhood,ATM,African Restaurant,Airport Lounge,Airport Service,American Restaurant,Apple Wine Pub,Argentinian Restaurant,Art Gallery,Art Museum,...,Transportation Service,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Well,Whisky Bar,Wine Bar,Wine Shop
0,Altstadt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Altstadt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,Altstadt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Altstadt,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Altstadt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
borough_onehot.shape

(867, 190)

In [16]:
pd.set_option('display.max_columns', None)
borough_grouped = borough_onehot.groupby('Neighbourhood').mean().reset_index()
borough_grouped

,Neighbourhood,ATM,African Restaurant,Airport Lounge,Airport Service,American Restaurant,Apple Wine Pub,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Austrian Restaurant,BBQ Joint,Bakery,Bank,Bar,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Boarding House,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Building,Burger Joint,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Chinese Restaurant,Chocolate Shop,Church,Cigkofte Place,Clothing Store,Cocktail Bar,Coffee Shop,College Residence Hall,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Currywurst Joint,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fishing Spot,Food & Drink Shop,Food Stand,Fountain,French Restaurant,Friterie,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Karaoke Bar,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Moroccan Restaurant,Motorcycle Shop,Museum,Music Store,Music Venue,Nightclub,Noodle House,Opera House,Organic Grocery,Other Nightlife,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Perfume Shop,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Post Office,Print Shop,Pub,Radio Station,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,River,Road,Sake Bar,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shopping Mall,Snack Place,Soccer Field,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Taverna,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Transportation Service,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Well,Whisky Bar,Wine Bar,Wine Shop
0,Altstadt,0.000000,0.00,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.030000,0.010000,0.01,0.000,0.010000,0.0,0.030000,0.010000,0.000000,0.010000,0.000000,0.010000,0.00,0.020000,0.01,0.00000,0.00,0.02,0.0,0.010000,0.000000,0.00,0.030000,0.000000,0.000000,0.01,0.000000,0.100000,0.000,0.000000,0.01,0.01,0.00,0.01,0.010000,0.020000,0.00000,0.00000,0.00,0.000000,0.00,0.000000,0.020000,0.010000,0.000000,0.0,0.000000,0.000000,0.01,0.010000,0.00000,0.01,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.010000,0.000000,0.00,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.060000,0.0000,0.01,0.000000,0.00000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.01,0.000000,0.00,0.010000,0.00,0.00,0.010000,0.000000,0.010000,0.00,0.00,0.00,0.010000,0.010000,0.00,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.00000,0.02,0.000000,0.010000,0.000000,0.000,0.000000,0.01,0.000000,0.000000,0.000000,0.0,0.010000,0.01,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.01,0.01,0.000000,0.00,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.010000,0.000000,0.000000,0.050000,0.010000,0.000000,0.000000,0.000000,0.000000,0.020000,0.01000,0.000000,0.010000,0.0000,0.000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.010000,0.010000,0.00,0.010000,0.000000,0.010000,0.000000,0.01,

In [17]:
borough_grouped.shape

(46, 190)

In [18]:
#url = 'https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data'
ffm_geo = "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/frankfurt-main.geojson"


folium.Choropleth(
    geo_data=ffm_geo,
    name='hotel',
    data=borough_grouped,
    columns=['Neighbourhood', 'Hotel'],
    key_on="feature.properties.name",
    fill_color='YlGn',
    fill_opacity=0.4,
    line_opacity=0.2,
    legend_name="Frequency of Hotels"
).add_to(map_ffm)

folium.Choropleth(
    geo_data=ffm_geo,
    name='trams',
    data=borough_grouped,
    columns=['Neighbourhood', 'Metro Station'],
    key_on="feature.properties.name",
    fill_color='RdPu',
    fill_opacity=0.4,
    line_opacity=0.2,
    legend_name="Frequency of Metro Stations"
).add_to(map_ffm)

folium.Choropleth(
    geo_data=ffm_geo,
    name='parks',
    data=borough_grouped,
    columns=['Neighbourhood', 'Park'],
    key_on="feature.properties.name",
    fill_color='YlOrBr',
    fill_opacity=0.4,
    line_opacity=0.2,
    legend_name="Frequency of Parks"
).add_to(map_ffm)

folium.LayerControl().add_to(map_ffm)

map_ffm

In [19]:
num_top_venues = 10

for hood in borough_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = borough_grouped[borough_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Altstadt----
               venue  freq
0               Café  0.10
1  German Restaurant  0.06
2         Restaurant  0.05
3              Plaza  0.05
4                Bar  0.03
5       Burger Joint  0.03
6         Art Museum  0.03
7        Men's Store  0.02
8     Scenic Lookout  0.02
9           Boutique  0.02


----Bahnhofsviertel----
                   venue  freq
0                  Hotel  0.10
1      Indian Restaurant  0.06
2                    Bar  0.06
3                 Bakery  0.04
4                   Café  0.04
5     Seafood Restaurant  0.04
6  Vietnamese Restaurant  0.03
7           Burger Joint  0.03
8     Chinese Restaurant  0.03
9            Coffee Shop  0.03


----Bergen-Enkheim----
                           venue  freq
0                          Plaza   1.0
1                            ATM   0.0
2  Paper / Office Supplies Store   0.0
3                         Museum   0.0
4                    Music Store   0.0
5                    Music Venue   0.0
6                    